In [ ]:
#@title
%%capture
!pip install gym pyvirtualdisplay
!apt-get install -y xvfb python-opengl ffmpeg
!apt-get install build-essential python-dev swig python-pygame
!apt-get update
!apt-get install cmake
!pip install --upgrade setuptools
!pip install ez_setup
!pip install gym[atari]
!pip install box2d-py

In [ ]:
#@title
%%capture

import gym
from gym.wrappers import Monitor
import glob
import io
import base64
from IPython.display import HTML, clear_output
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment 
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [ ]:
import numpy as np
import gym
from matplotlib import pyplot as plt

In [ ]:
class Net():
  def __init__(self, chromosome):
    # 8 => 10 => 4
    self.w1 = chromosome[:80].reshape((10,8))
    self.b1 = chromosome[80:90].reshape((10,))
    self.w2 = chromosome[90:130].reshape((4,10))
    self.b2 = chromosome[130:].reshape((4,))
  def forward(self, x):
    s1 = np.dot(self.w1, x) + self.b1
    z1 = s1 * (s1 > 0.0)
    s2 = np.dot(self.w2, z1) + self.b2
    z2 = s2 * (s2 > 0.0)
    return z2.argmax()

In [ ]:
def create_individual():
  ''' Esta função cria o genótipo de um indivíduo
      completamente aleatório
  '''
  N = 8*10+10+10*4+4
  chromosome = np.random.normal(0, 10, size=(N,))
  return chromosome, None

In [ ]:
def create_population(size):
  ''' Aqui geramos uma população inteira
  '''
  # Nossa população será uma lista
  population = list()
  # Nesse laço criamos cada indivíduo
  # e adicionamos à população
  for _ in range(size):
    population.append(create_individual())
  return population

In [ ]:
def get_phenotype(chromosome):
  ''' Esta função gera a imagem a partir do
      gene.
  '''
  n = Net(chromosome)
  return n

In [ ]:
def fitness(n):
  R = 0.0
  trials = 6
  env = gym.make('LunarLander-v2')
  for _ in range(trials):
    obs = env.reset()
    while True:
      action = n.forward(obs)
      obs, reward, done, info = env.step(action)
      R += reward
      if done:
        break
  env.close()
  return (10000.0 + R/trials) / 10000.0

In [ ]:
def crossover2point(chr1, chr2):
  ''' Esta função combina os genes de
      dois indivíduos, gerando um novo
      cromossomo
  '''
  N = len(chr1)
  idx1 = np.random.randint(0,N)
  idx2 = np.random.randint(0,N)
  if idx1 > idx2:
    idx1, idx2 = idx2, idx1
  new_chr1 = np.concatenate((chr1[0:idx1], chr2[idx1:idx2], chr1[idx2:]))
  new_chr2 = np.concatenate((chr2[0:idx1], chr1[idx1:idx2], chr2[idx2:]))
  return new_chr1, new_chr2

In [ ]:
def mutation(chr, p):
  ''' Esta função aplica a mutação em um dos
      genes de nosso cromossomo
  '''
  N = len(chr)
  total = int(np.random.normal(p*N, p*N))
  if total < 0:
    total = 0
  elif total > N:
    total = N
  for _ in range(total):
    idx = np.random.randint(0, N)
    if np.random.rand() > 0.5:
      chr[idx] += np.random.normal(0, 1)
    else:
      chr[idx] = np.random.normal(0, 10)
  return chr

In [ ]:
def compute_fitness(population):
  ''' Esta função avalia cada indivíduo
      e adiciona o seu score.
  '''
  scored = list()
  for chromosome, score in population:
    n = get_phenotype(chromosome)
    score = fitness(n)
    scored.append((chromosome, score))
  scored.sort(key=lambda x:x[1], reverse=True)
  return scored

In [ ]:
def roulette(population):
  fitnesses = np.array([np.exp(fitness) \
                       for _, fitness in population])
  total = np.sum(fitnesses)
  fitnesses /= total
  choice = np.random.rand()
  subtotal = 0
  idx = 0
  for f in fitnesses:
    subtotal += f
    if subtotal >= choice or idx == len(fitnesses)-1:
      break
    idx += 1
  return idx

In [ ]:
def new_generation(population, size, mutation_rate):
  new_population = list()
  while len(new_population) < size:
    idx1 = roulette(population)
    idx2 = roulette(population)
    if idx1 == idx2:
      continue
    chr1, _ = population[idx1]
    chr2, _ = population[idx2]
    new_chr1, new_chr2 = crossover2point(chr1, chr2)
    new_chr1 = mutation(new_chr1, mutation_rate)
    new_chr2 = mutation(new_chr2, mutation_rate)
    new_population.append((new_chr1, None))
    new_population.append((new_chr2, None))
  return new_population[:size]

In [ ]:
def genalg(pop_size, elite_size, \
           epochs, mutation_rate):
  x = list()
  y = list()

  population = create_population(pop_size)

  for generation in range(epochs+1):

    population = compute_fitness(population)

    chr, fit = population[0]
    _, worst = population[-1]
    n = get_phenotype(chr)
    y.append(fit)
    x.append(generation)

    R = 0.0
    env2 = wrap_env(gym.make('LunarLander-v2'))
    obs = env2.reset()
    while True:
      action = n.forward(obs)
      obs, reward, done, info = env2.step(action)
      env2.render()
      R += reward
      if done:
        break
    env2.close()

    clear_output()
    print('Geração', generation, ', melhor:', fit, ', pior', worst)
    show_video()
      
    plt.plot(x, y)
    plt.show()

    elite = population[:elite_size]
    population = new_generation(population, pop_size - elite_size,\
                                mutation_rate) + elite
  return n, x, y

In [ ]:
#n, x, y = genalg(50, 3, 1000, 0.03)

In [ ]:
env = wrap_env(gym.make('LunarLander-v2'))

R = 0.0
obs = env.reset()
while True:
  action = np.random.randint(0,4)
  obs, reward, done, info = env.step(action)
  env.render()
  R += reward
  if done:
    break

print(R)

env.close()
show_video()

-77.16412163723079
